In [5]:
! pip install python-docx langchain-community unstructured sentence_transformers

In [6]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load tài liệu từ file Word (.docx)
loader = UnstructuredWordDocumentLoader("data.docx")
documents = loader.load()

# Chia nhỏ tài liệu thành các đoạn ngắn
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = splitter.split_documents(documents)


In [12]:
import os
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

# Tắt xác minh SSL (chỉ dùng khi cần thiết)
os.environ['HF_HUB_DISABLE_SSL_VERIFICATION'] = '1'

# Tải mô hình embedding
model = SentenceTransformer('all-MiniLM-L6-v2')

# Chuyển văn bản thành vector
texts = [doc.page_content for doc in docs]
embeddings = model.encode(texts)

# Tạo đối tượng Document cho FAISS
documents = [Document(page_content=text) for text in texts]

# Tạo FAISS vector store
vector_db = FAISS.from_embeddings(embeddings, documents)

# Lưu FAISS vào thư mục để dùng lại
vector_db.save_local("faiss_index")


No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with mean pooling.


SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/adapter_config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))"), '(Request ID: 48585288-0fba-430d-86e7-87c7e304a57e)')

In [ ]:
# Load lại FAISS index
# vector_db = FAISS.load_local("faiss_index", embedding_model)

# Truy vấn người dùng
query = "Chính sách bảo hành sản phẩm như thế nào?"
retrieved_docs = vector_db.similarity_search(query, k=3)

# Ghép nội dung truy xuất thành prompt
context = "\n\n".join([doc.page_content for doc in retrieved_docs])
prompt = f"""Dựa trên thông tin sau, hãy trả lời câu hỏi:

{context}

Câu hỏi: {query}
"""


ValueError: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).

In [16]:
import requests

url = "http://172.31.98.63:5678/webhook-test/my-api-trigger"
data = {"message": "Hello from Python"}

response = requests.post(url, json=data)
print(response.text)


{"code":404,"message":"The requested webhook \"my-api-trigger\" is not registered.","hint":"Click the 'Execute workflow' button on the canvas, then try again. (In test mode, the webhook only works for one call after you click this button)","stacktrace":"ResponseError: The requested webhook \"my-api-trigger\" is not registered.\n    at TestWebhooks.getWebhookMethods (C:\\Users\\loitq\\AppData\\Roaming\\npm\\node_modules\\n8n\\src\\webhooks\\test-webhooks.ts:205:37)\n    at TestWebhooks.executeWebhook (C:\\Users\\loitq\\AppData\\Roaming\\npm\\node_modules\\n8n\\src\\webhooks\\test-webhooks.ts:81:22)\n    at WebhookRequestHandler.handleRequest (C:\\Users\\loitq\\AppData\\Roaming\\npm\\node_modules\\n8n\\src\\webhooks\\webhook-request-handler.ts:48:21)\n    at C:\\Users\\loitq\\AppData\\Roaming\\npm\\node_modules\\n8n\\src\\webhooks\\webhook-request-handler.ts:143:3"}
